In [12]:
import numpy as np 
from tensorflow.python.framework import ops
from sklearn.metrics.pairwise import pairwise_distances 
from sklearn.neighbors import kneighbors_graph
import matplotlib.pyplot as plt
import scipy.io 
import tensorflow as tf
from matplotlib.colors import ListedColormap
from tfdeterminism import patch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report 
patch()

TensorFlow version 1.14.0 has been patched using tfdeterminism version 0.3.0


In [13]:
import numpy as np
from sklearn.neighbors import KDTree
from scipy.sparse import csr_matrix

def knn_adjacency_matrix(ALL_X, k=10):
    # Build a KDTree to efficiently compute KNN for each point
    tree = KDTree(ALL_X)

    # Initialize the adjacency matrix with zeros
    N = ALL_X.shape[0]
    A = np.zeros((N, N))

    # Compute the KNN graph for each point
    for i in range(N):
        _, indices = tree.query(ALL_X[i:i+1, :], k+1)
        indices = indices[0, 1:]  # exclude self-connection
        A[i, indices] = 1

    # Convert the graph to a sparse matrix and return it
    ALL_L = csr_matrix(A)
    return ALL_L


In [14]:
def split_data(Labeled_points, Labels, test_size=0.3, random_state=0):
     

    # Set the random seed, if provided
    if random_state is not None:
        np.random.seed(random_state)

    # Get the number of data points
    num_points = Labeled_points.shape[0]

    # Calculate the number of points to use for testing
    num_test = int(num_points * test_size)

    # Shuffle the indices of the data points
    indices = np.random.permutation(num_points)

    # Split the indices into training and testing sets
    test_indices = indices[:num_test]
    train_indices = indices[num_test:]

    # Use the indices to get the training and testing data points and labels
    X_train = Labeled_points[train_indices]
    X_test = Labeled_points[test_indices]
    y_train = Labels[train_indices]
    y_test = Labels[test_indices]

    return X_train, X_test, y_train, y_test


In [15]:
def create_placeholders(n_x, n_y):

    x_in = tf.placeholder(tf.float32,  [None, n_x], name = "x_in")
    y_in = tf.placeholder(tf.float32, [None, n_y], name = "y_in")
    lap = tf.placeholder(tf.float32, [None, None], name = "lap")
    isTraining = tf.placeholder_with_default(True, shape=())
    
    return x_in, y_in, lap,isTraining


In [16]:
def initialize_parameters():
   
    tf.set_random_seed(1)

    x_w1 = tf.get_variable("x_w1", [224,118], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    x_b1 = tf.get_variable("x_b1", [118], initializer = tf.zeros_initializer())
     
    x_w2 = tf.get_variable("x_w2", [118,12], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    x_b2 = tf.get_variable("x_b2", [12], initializer = tf.zeros_initializer())
    
    
    parameters = {"x_w1": x_w1,
                  "x_b1": x_b1,
                  "x_w2": x_w2,
                  "x_b2": x_b2}
                  
    return parameters


In [17]:
def GCN_layer(x_in, L_, weights): 
    x_mid = tf.matmul(x_in, weights)
    x_out = tf.matmul(L_, x_mid)
    
    return x_out

In [18]:
def mynetwork(x, parameters, Lap, isTraining, momentums = 0.9):

    with tf.name_scope("x_layer_1"):
        
         x_z1_bn = tf.layers.batch_normalization(x, momentum = momentums, training = isTraining)          
         x_z1 = GCN_layer(x_z1_bn, Lap, parameters['x_w1']) + parameters['x_b1']
         x_z1_bn = tf.layers.batch_normalization(x_z1, momentum = momentums, training = isTraining)   
         x_a1 = tf.nn.relu(x_z1_bn)     

    with tf.name_scope("x_layer_2"):
         
         x_z2_bn = tf.layers.batch_normalization(x_a1, momentum = momentums, training = isTraining)            
         x_z2 = GCN_layer(x_z2_bn, Lap, parameters['x_w2']) + parameters['x_b2']         
         
    l2_loss =  tf.nn.l2_loss(parameters['x_w1']) + tf.nn.l2_loss(parameters['x_w2'])
                
    return x_z2, l2_loss

In [19]:
def mynetwork_optimaization(y_est, y_re, l2_loss, reg, learning_rate, global_step):
    
    with tf.name_scope("cost"):
     #     cost = (tf.nn.softmax_cross_entropy_with_logits(logits = y_est, labels = y_re)) 
         cost = (tf.nn.softmax_cross_entropy_with_logits(logits = y_est, labels = y_re)) +  reg * l2_loss
         cost = tf.reduce_mean(cost) +  reg * l2_loss
         
    with tf.name_scope("optimization"):
         update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
         optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost,  global_step=global_step)
         optimizer = tf.group([optimizer, update_ops])
         
    return cost, optimizer 

In [20]:
def Accuracy(preds, labels):

      correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
      accuracy = tf.cast(correct_prediction, "float")  
      
      return tf.reduce_mean(accuracy)

In [21]:
def train_mynetwork(X_train, X_test, Y_train, Y_test, train_L ,test_L, learning_rate = 0.001, beta_reg = 0.001, num_epochs = 200, print_cost = True):
                                    #GCN_mask_TR, GCN_mask_TE
    ops.reset_default_graph()                                                         
    (m, n_x) = X_train.shape             # n_x=200
    (m, n_y) = Y_train.shape      
    
    costs = []                                        
    costs_dev = []
    train_acc = []
    val_acc = []
    
    x_in, y_in, lap, isTraining = create_placeholders(n_x, n_y) 
    
    parameters = initialize_parameters()
    with tf.name_scope("network"):
        x_out, l2_loss = mynetwork(x_in, parameters, lap, isTraining)
    global_step = tf.Variable(0, trainable=False)
    
    with tf.name_scope("optimization"):
         cost, optimizer = mynetwork_optimaization(x_out, y_in, l2_loss, beta_reg, learning_rate, global_step)

    with tf.name_scope("metrics"):
         accuracy_train = Accuracy(x_out, y_in)
         accuracy_test= Accuracy(x_out, y_in)
         
    init = tf.global_variables_initializer()
   
    with tf.Session() as sess:
        
        sess.run(init)
        # Do the training loop
        for epoch in range(num_epochs + 1):

            _, epoch_cost, epoch_acc = sess.run([optimizer, cost, accuracy_train], feed_dict={x_in: X_train, y_in: Y_train, lap: train_L, isTraining: True})
            
            
            features, overall_cost_dev, overall_acc_dev = sess.run([x_out, cost, accuracy_test], feed_dict={x_in: X_test, y_in: Y_test, lap: test_L, isTraining: False})
                # print ("epoch %i: Train_loss: %f, Val_loss: %f, Train_acc: %f, Val_acc: %f" % (epoch, epoch_cost, overall_cost_dev, epoch_acc, overall_acc_dev))
            
            
            costs.append(epoch_cost)
            train_acc.append(epoch_acc)
            costs_dev.append(overall_cost_dev)
            val_acc.append(overall_acc_dev)
        print("Training")
        for i in range(5):
            print(f"epoch {i*50}: Train_loss: {costs[i*50]}, Train_acc: {train_acc[i*50]}") 
        print("Validating")
        for j in range(5):
            print(f"epoch {j*50}: Validating_loss: {costs_dev[j*50]}, Validating_acc: {val_acc[j*50]}")

            
        # plot the cost      
        plt.plot(np.squeeze(costs), label='Training cost')
        plt.plot(np.squeeze(costs_dev), label='Validation cost')
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
        # plot the accuracy 
        plt.plot(np.squeeze(train_acc), label='Training accuracy')
        plt.plot(np.squeeze(val_acc), label='Validation accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
    
        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")
    
        return parameters , val_acc, features
     



Training
epoch 0: Train_loss: 21.434829711914062, Train_acc: 0.039402175694704056
epoch 50: Train_loss: 0.8021205067634583, Train_acc: 0.7554348111152649
epoch 100: Train_loss: 0.6612743735313416, Train_acc: 0.8125
epoch 150: Train_loss: 0.6145084500312805, Train_acc: 0.8260869383811951
epoch 200: Train_loss: 0.5852092504501343, Train_acc: 0.8383151888847351
Validating
epoch 0: Validating_loss: 13.500717163085938, Validating_acc: 0.13650794327259064
epoch 50: Validating_loss: 2.1829311847686768, Validating_acc: 0.40634921193122864
epoch 100: Validating_loss: 1.7172889709472656, Validating_acc: 0.5460317730903625
epoch 150: Validating_loss: 1.8169097900390625, Validating_acc: 0.5428571701049805
epoch 200: Validating_loss: 1.8929287195205688, Validating_acc: 0.5396825671195984


Parameters have been trained!
(315,) (315,)
Confusion Matrix:
 [[15  0  0  1  0  7  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  2  6  7  2  0]
 [ 0  2  0  0  0  0  0  3  1  0  0  0]
 [ 0  0  0  6  0  5  0  5  0  0  0  0]
 [ 0  5  0  0 24  0  0  0 11  0  0  0]
 [ 0  0  0  0  0  6  0  3  4  0  0  0]
 [ 0  2  0  2  0  3  0  2  2  0  0  5]
 [ 0 10  0  0  0  0  0 10  6  0  0  0]
 [ 0  7  0  0  0  0  0  5 50  0  0  0]
 [ 0  9  0  0  0  0  0  0  1 32  5  0]
 [ 0  2  0  0  0  0  0  0  0 15 16  0]
 [ 0  2  0  2  0  1  0  0  0  0  0  1]]
Accuracy: 0.5396825396825397
Precision: 0.4523160139964833
Recall: 0.41520966315132174
F1-score: 0.41817794392703195
ROC AUC score: 0.842587158916776


In [22]:
import scipy.sparse as sp
import warnings
warnings.filterwarnings('ignore')
 
 
Labeled_points = scipy.io.loadmat('Labeled_Cuprite_Data/data_points.mat')['data_points']
maxVal = np.amax(Labeled_points)
minVal = np.amin(Labeled_points) 
Labeled_points_shifted = Labeled_points + abs(minVal) 
Labeled_points_norm = Labeled_points_shifted / (abs(minVal) + abs(maxVal))
Labeled_points_norm = Labeled_points_norm + 1e-6 

Labels = scipy.io.loadmat('Labeled_Cuprite_Data/labels.mat')['labels']
Labels = Labels.T
num_classes = 12
Labels_onehot = tf.keras.utils.to_categorical(Labels, num_classes) 

X_train, X_test, Y_train, Y_test = split_data(Labeled_points_norm, Labels_onehot, test_size=0.3) 

 
train_L = knn_adjacency_matrix(X_train)
test_L = knn_adjacency_matrix(X_test)


  
parameters, val_acc, features = train_mynetwork(X_train, X_test, Y_train, Y_test, train_L.todense() ,test_L.todense()) 
y_pred = np.argmax(features, axis=1)
y_true = np.argmax(Y_test, axis=1) 

print(y_pred.shape, y_true.shape)
# create confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print confusion matrix
print("Confusion Matrix:\n", cm)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1-score:", f1_score(y_true, y_pred, average='macro'))
print("ROC AUC score:", roc_auc_score(Y_test, features, multi_class='ovr'))
